In [1]:
# %%
# === IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
import joblib
import os

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
sns.set_style("whitegrid")

print("✅ Imports OK")

# %%
# === CHARGEMENT TEST ===
print("="*60)
print("CHARGEMENT DU TEST SET")
print("="*60)

df = pd.read_csv("../data/interim/test_dataset_M1_interim.csv")
test_ids = df['id'].copy()
df = df.drop(columns=['id'])

print(f"Dataset shape: {df.shape}")
print(f"Test IDs: {len(test_ids)}")

# %%
# === FEATURE ENGINEERING - PRIX ===
print("\n" + "="*60)
print("FEATURE ENGINEERING - Prix")
print("="*60)

df["Net_Price"] = df["Price"] * (1 - df["Discount"] / 100)
df["Price_Bucket"] = pd.qcut(df["Price"], 5, labels=False, duplicates="drop")

print("✅ Features prix créées")

# %%
# === FEATURE ENGINEERING - ENGAGEMENT ===
print("\n" + "="*60)
print("FEATURE ENGINEERING - Engagement")
print("="*60)

df["Email_x_Engagement"] = df["Email_Interaction"] * df["Engagement_Score"]
df["Cart_x_Engagement"] = df["Items_In_Cart"] * df["Engagement_Score"]

print("✅ Features engagement créées")

# %%
# === FEATURE ENGINEERING - SEGMENTS ===
print("\n" + "="*60)
print("FEATURE ENGINEERING - Segments")
print("="*60)

df["Campaign_Period"] = df["Campaign_Period"].astype(bool)
df["HighValue_Category"] = df["Category"].isin([0.0, 1.0, 2.0]).astype(int)

print("✅ Features segments créées")

# %%
# === 🚀 INTERACTIONS CIBLÉES ===
print("\n" + "="*60)
print("🚀 INTERACTIONS QUADRATIQUES CIBLÉES")
print("="*60)

df["Items_Cart_Squared"] = df["Items_In_Cart"] ** 2
df["Engagement_Squared"] = df["Engagement_Score"] ** 2
df["Cart_per_Review"] = df["Items_In_Cart"] / (df["Reviews_Read"] + 1)
df["Engagement_per_Price"] = df["Engagement_Score"] / (df["Net_Price"] + 1)
df["Engagement_x_Campaign"] = df["Engagement_Score"] * df["Campaign_Period"].astype(int)
df["Price_x_Campaign"] = df["Net_Price"] * df["Campaign_Period"].astype(int)

print(f"✅ 6 interactions ciblées créées")

# %%
# === MISSING INDICATORS ===
print("\n" + "="*60)
print("MISSING INDICATORS")
print("="*60)

missing_pct = df.isnull().sum() / len(df) * 100
cols_with_missing = missing_pct[missing_pct > 1].index.tolist()
cols_with_missing = [col for col in cols_with_missing if col not in ['Day']]

for col in cols_with_missing:
    df[f"{col}_missing"] = df[col].isnull().astype(int)

print(f"✅ {len(cols_with_missing)} missing indicators créés")

# %%
# === IMPUTATION ===
print("\n" + "="*60)
print("IMPUTATION")
print("="*60)

median_values = {
    'Age': 30.0, 'Gender': 0.0, 'Reviews_Read': 3.0,
    'Price': 300.0, 'Discount': 25.0, 'Category': 2.0,
    'Items_In_Cart': 3.0, 'Socioeconomic_Status_Score': 4.5,
    'Engagement_Score': 2.5, 'AB_Bucket': 3.0,
    'Price_Sine': 0.0, 'Day': 80.0, 'Email_Interaction': 0.0,
}

numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        fill_value = median_values.get(col, df[col].median())
        df[col].fillna(fill_value, inplace=True)

categorical_cols = df.select_dtypes(include=['object', 'bool']).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna("Unknown", inplace=True)

print(f"✅ Imputation terminée")

# %%
# === RARE CATEGORIES ===
print("\n" + "="*60)
print("RARE CATEGORIES")
print("="*60)

threshold = 0.01
for col in categorical_cols:
    if col == 'Campaign_Period':
        continue
    value_counts = df[col].value_counts(normalize=True)
    rare_categories = value_counts[value_counts < threshold].index.tolist()
    if rare_categories:
        df[col] = df[col].apply(lambda x: "Other" if x in rare_categories else x)

print("✅ Rare categories regroupées")

# %%
# === 🧹 NETTOYAGE RADICAL ===
print("\n" + "="*60)
print("🧹 NETTOYAGE RADICAL")
print("="*60)

low_value_features = [
    'Socioeconomic_Status_Score', 'AB_Bucket', 'Discount',
    'Age', 'Gender', 'Referral_Source', 'Payment_Method',
    'Time_of_Day', 'Price_Sine', 'Email_Interaction',
]

features_removed = []
for feat in low_value_features:
    if feat in df.columns:
        df = df.drop(columns=[feat])
        features_removed.append(feat)

print(f"✅ {len(features_removed)} features supprimées")
print(f"📊 Features restantes: {len(df.columns)}")

# %%
# === PIPELINE CATBOOST ===
print("\n" + "="*60)
print("PIPELINE CATBOOST")
print("="*60)

cat_features = ["Device_Type", "Category"]

for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype(str)

cols_to_drop_cb = ["Session_ID"]
X_test_cb = df.drop(columns=cols_to_drop_cb, errors='ignore')

cat_indices = [X_test_cb.columns.get_loc(col) for col in cat_features if col in X_test_cb.columns]

print(f"✅ Pipeline CatBoost préparé:")
print(f"   X_test: {X_test_cb.shape}")

# %%
# === PIPELINE CLASSIC ===
print("\n" + "="*60)
print("PIPELINE CLASSIC")
print("="*60)

try:
    train_data = joblib.load("../data/processed/classic_ready.pkl")
    train_columns = train_data["X_train"].columns.tolist()
    print(f"✅ Colonnes train chargées: {len(train_columns)} features")
except:
    print("⚠️ classic_ready.pkl non trouvé")
    train_columns = None

df_classic = df.copy()
df_classic["Campaign_Period"] = df_classic["Campaign_Period"].astype(int)

cols_onehot = ["Device_Type", "Category"]
df_classic = pd.get_dummies(df_classic, columns=cols_onehot, drop_first=True)

cols_to_drop = ["Session_ID"]
X_test_cls = df_classic.drop(columns=cols_to_drop, errors='ignore')

if train_columns is not None:
    missing_cols = set(train_columns) - set(X_test_cls.columns)
    for col in missing_cols:
        X_test_cls[col] = 0
    
    extra_cols = set(X_test_cls.columns) - set(train_columns)
    if extra_cols:
        X_test_cls = X_test_cls.drop(columns=list(extra_cols))
    
    X_test_cls = X_test_cls[train_columns]

scaler = StandardScaler()
X_test_cls_scaled = scaler.fit_transform(X_test_cls)
X_test_cls = pd.DataFrame(X_test_cls_scaled, columns=X_test_cls.columns)

print(f"✅ Pipeline Classic préparé:")
print(f"   X_test: {X_test_cls.shape}")

# %%
# === SAUVEGARDE ===
print("\n" + "="*60)
print("SAUVEGARDE")
print("="*60)

os.makedirs("../data/processed", exist_ok=True)

catboost_data = {
    "X_test": X_test_cb,
    "test_ids": test_ids,
    "cat_indices": cat_indices,
    "cat_features_names": [col for col in X_test_cb.columns if col in cat_features],
}

joblib.dump(catboost_data, "../data/processed/catboost_ready_test.pkl", protocol=4)
print("✅ CatBoost test: 'catboost_ready_test.pkl'")

classic_data = {
    "X_test": X_test_cls,
    "test_ids": test_ids,
    "scaler": scaler,
}

joblib.dump(classic_data, "../data/processed/classic_ready_test.pkl", protocol=4)
print("✅ Classic test: 'classic_ready_test.pkl'")

print("\n" + "="*60)
print("🎉 TEST SET OPTIMISÉ TERMINÉ !")
print("="*60)
print(f"📦 Features: {X_test_cb.shape[1]} (vs 64 avant)")
print("="*60)


✅ Imports OK
CHARGEMENT DU TEST SET
Dataset shape: (6145, 19)
Test IDs: 6145

FEATURE ENGINEERING - Prix
✅ Features prix créées

FEATURE ENGINEERING - Engagement
✅ Features engagement créées

FEATURE ENGINEERING - Segments
✅ Features segments créées

🚀 INTERACTIONS QUADRATIQUES CIBLÉES
✅ 6 interactions ciblées créées

MISSING INDICATORS
✅ 24 missing indicators créés

IMPUTATION
✅ Imputation terminée

RARE CATEGORIES
✅ Rare categories regroupées

🧹 NETTOYAGE RADICAL
✅ 10 features supprimées
📊 Features restantes: 44

PIPELINE CATBOOST
✅ Pipeline CatBoost préparé:
   X_test: (6145, 43)

PIPELINE CLASSIC
✅ Colonnes train chargées: 47 features
✅ Pipeline Classic préparé:
   X_test: (6145, 47)

SAUVEGARDE
✅ CatBoost test: 'catboost_ready_test.pkl'
✅ Classic test: 'classic_ready_test.pkl'

🎉 TEST SET OPTIMISÉ TERMINÉ !
📦 Features: 43 (vs 64 avant)
